In [1]:
import pandas as pd

from sklearn.cluster import DBSCAN
import sklearn.utils
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

In [2]:
data_df = pd.read_csv('Kaagle_Upload.csv')
data_df.head()

C:\Users\Muskan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,accident_index,vehicle_reference,vehicle_type,towing_and_articulation,vehicle_manoeuvre,vehicle_location-restricted_lane,junction_location,skidding_and_overturning,hit_object_in_carriageway,vehicle_leaving_carriageway,...,age_band_of_casualty,casualty_severity,pedestrian_location,pedestrian_movement,car_passenger,bus_or_coach_passenger,pedestrian_road_maintenance_worker,casualty_type,casualty_home_area_type,casualty_imd_decile
0,201597UD71807,1,9,0,7,0,2,1,0,0,...,4.0,3.0,5.0,1.0,0.0,0.0,0.0,0.0,3.0,-1.0
1,201597UD71607,1,9,0,7,0,4,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201597UD71607,2,9,0,8,0,4,0,0,0,...,8.0,3.0,0.0,0.0,2.0,0.0,0.0,9.0,3.0,-1.0
3,201597UD71412,1,9,0,17,0,0,2,0,1,...,5.0,3.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0,-1.0
4,201597UD71407,1,9,0,17,0,0,0,0,1,...,1.0,3.0,0.0,0.0,1.0,0.0,0.0,9.0,1.0,-1.0


In [3]:
data_df.shape

(172651, 70)

In [4]:
data_df.drop([  'casualty_reference', 'casualty_class',
       'sex_of_casualty', 'age_of_casualty', 'age_band_of_casualty',
       'casualty_severity', 'pedestrian_location', 'pedestrian_movement',
       'car_passenger', 'bus_or_coach_passenger',
       'pedestrian_road_maintenance_worker', 'casualty_type',
       'casualty_home_area_type', 'casualty_imd_decile','urban_or_rural_area', 'did_police_officer_attend_scene_of_accident',
             'pedestrian_crossing-human_control', 'junction_control',
             'number_of_vehicles','number_of_casualties','accident_severity',
             'location_easting_osgr', 'location_northing_osgr',
             'NUmber_of_Casualities_unique_to_accident_index',
              'No_of_Vehicles_involved_unique_to_accident_index','driver_imd_decile', 'driver_home_area_type',
              'vehicle_imd_decile','engine_capacity_(cc)', 'propulsion_code','age_of_driver','hit_object_in_carriageway',
              'vehicle_leaving_carriageway', 'hit_object_off_carriageway',
              '1st_point_of_impact','journey_purpose_of_driver','vehicle_reference','towing_and_articulation', 'lsoa_of_accident_location'],axis=1,inplace=True )

In [5]:
data_df['date'] = pd.to_datetime(data_df['date'], format='%d/%m/%Y')

In [6]:
data_df['year'] = data_df['date'].dt.year
data_df['month'] = data_df['date'].dt.month
data_df['day'] = data_df['date'].dt.day

In [7]:
data_df.drop(['date'],axis=1,inplace=True)
data_df.head()

,accident_index,vehicle_type,vehicle_manoeuvre,vehicle_location-restricted_lane,junction_location,skidding_and_overturning,was_vehicle_left_hand_drive?,sex_of_driver,age_band_of_driver,age_of_vehicle,...,2nd_road_number,pedestrian_crossing-physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,year,month,day
0,201597UD71807,9,7,0,2,1,1,1,8,7,...,719,0,1,1,1,0,0,2015,7,15
1,201597UD71607,9,7,0,4,0,1,2,9,8,...,78,0,1,1,1,0,0,2015,7,28
2,201597UD71607,9,8,0,4,0,1,1,8,10,...,78,0,1,1,1,0,0,2015,7,28
3,201597UD71412,9,17,0,0,2,1,1,5,14,...,0,0,6,3,4,0,0,2015,12,31
4,201597UD71407,9,17,0,0,0,1,2,5,9,...,0,1,1,2,2,0,0,2015,7,26


In [8]:
list(data_df.columns.values)

['accident_index',
 'vehicle_type',
 'vehicle_manoeuvre',
 'vehicle_location-restricted_lane',
 'junction_location',
 'skidding_and_overturning',
 'was_vehicle_left_hand_drive?',
 'sex_of_driver',
 'age_band_of_driver',
 'age_of_vehicle',
 'longitude',
 'latitude',
 'police_force',
 'day_of_week',
 'time',
 'local_authority_(district)',
 'local_authority_(highway)',
 '1st_road_class',
 '1st_road_number',
 'road_type',
 'speed_limit',
 'junction_detail',
 '2nd_road_class',
 '2nd_road_number',
 'pedestrian_crossing-physical_facilities',
 'light_conditions',
 'weather_conditions',
 'road_surface_conditions',
 'special_conditions_at_site',
 'carriageway_hazards',
 'year',
 'month',
 'day']

In [9]:
data_df['time'] = pd.to_datetime(data_df['time'], format=' %H:%M:%S')
data_df['Hour'] = data_df['time'].dt.hour

In [10]:
data_df.drop(['time'], axis=1, inplace=True)

In [11]:
data_df.insert(0, 'accident', 1)

In [12]:
data_df.drop(['accident_index'], axis = 1, inplace = True)

In [14]:
data_df = data_df.dropna()

In [15]:
data_df_clus = data_df[["latitude","longitude"]]
#data_df_clus = StandardScaler().fit_transform(data_df_clus)

In [16]:
db = DBSCAN(eps=0.03, min_samples=15).fit(data_df_clus)
labels = db.labels_
print(labels[4000:4560])

[ 8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8

In [17]:
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
clusterNum= len(set(labels))
#print(clusterNum)
print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

#print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(data_df_clus, labels))


Estimated number of clusters: 427
Estimated number of noise points: 13515


In [19]:
data_temp =pd.DataFrame(data_df_clus).copy()
data_temp["cluster"] = labels

In [20]:
data_temp.to_csv('clusters.csv', index=False)